https://medium.com/swlh/retrieving-full-historical-data-for-every-cryptocurrency-on-binance-bitmex-using-the-python-apis-27b47fd8137f

In [3]:
import pandas as pd
import math
import os.path
import time
from bitmex import bitmex
from datetime import timedelta, datetime
from dateutil import parser
from tqdm import tqdm_notebook #(Optional, used for progress-bars)
import tqdm
import backtrader as bt
from ccxtbt import CCXTStore, CCXTFeed
import backtrader as bt
from datetime import datetime, timedelta
import json
import ccxt
import pprint

In [4]:
### API
bitmex_api_key = 'PUNPhX0cNSinQ7Zv4TRAETdx'    #Enter your own API-key here
bitmex_api_secret = 'duc5b9JxLy68Oza_9cfv6I8HztwaYFSOdETWmOYOCojFtGv7' #Enter your own API-secret here

In [5]:
### CONSTANTS
binsizes = {"1m": 1, "5m": 5, "1h": 60, "1d": 1440}
batch_size = 750
bitmex_client = bitmex(test=True, api_key=bitmex_api_key, api_secret=bitmex_api_secret)

/home/james/anaconda3/lib/python3.8/site-packages/swagger_spec_validator/validator20.py:49: SwaggerValidationWarning: Found "$ref: #/definitions/UserPreferences" with siblings that will be overwritten. See https://stackoverflow.com/a/48114924 for more information. (path #/definitions/User/properties/preferences)
  warnings.warn(


In [11]:

def get_all_bitmex(symbol, kline_size, start_datetime, end_datetime, save = False):
    filename = 'Fa%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): 
        data_df = pd.read_csv(filename)
        return data_df
    else: 
        data_df = pd.DataFrame()
        
    delta_min = (end_datetime - start_datetime).total_seconds() / 60 # number of mins
    
    available_data = math.ceil(delta_min/binsizes[kline_size])
    
    
    rounds = math.ceil(available_data / batch_size)
    
    if rounds > 0:
        print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data in %d rounds.' % (delta_min, symbol, available_data, kline_size, rounds))
        for round_num in tqdm_notebook(range(rounds)):
            time.sleep(1)
            new_time = (start_datetime + timedelta(minutes = round_num * batch_size * binsizes[kline_size]))
            
            data = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=batch_size, startTime = new_time).result()[0]
            
            temp_df = pd.DataFrame(data)
            data_df = data_df.append(temp_df)
            
    data_df["time"] = data_df["timestamp"].dt.time
    data_df["timestamp"] = data_df["timestamp"].astype(str).str[:-6]
    data_df = data_df.rename(columns={'timestamp': 'datetime', "vwap": "openinterest"})
    data_df.set_index('datetime', inplace=True)
    data_df = data_df.drop(["symbol", "trades"],axis=1)
    
    if save and rounds > 0: 
        data_df.to_csv(filename)
    print('All caught up..!')
    
    return data_df

In [12]:
start_datetime = datetime(2020, 1, 1, 0, 0, 0, 0)
end_datetime = datetime(2020, 6, 30, 23, 59, 59, 0)


bitmex_symbols = ["XBTUSD"]
datas = []
for symbol in bitmex_symbols:
    datas.append(get_all_bitmex(symbol, '1m', start_datetime, end_datetime, save = True))

<ipython-input-11-90c7f4c3ed9c>:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for round_num in tqdm_notebook(range(rounds)):



All caught up..!


In [137]:
datas[0]

,open,high,low,close,volume,openinterest,lastSize,turnover,homeNotional,foreignNotional,time
datetime,,,,,,,,,,,
2020-05-25 00:00:00,8599.0,8599.0,8565.5,8582.0,69877,8566.7780,100.0,815689104,8.156891,69877.0,00:00:00
2020-05-25 00:01:00,8582.0,8581.5,8533.0,8533.0,43479,8552.8567,2.0,508377808,5.083778,43479.0,00:01:00
2020-05-25 00:02:00,8533.0,8534.5,8532.0,8534.0,2629,8534.6078,881.0,30806573,0.308066,2629.0,00:02:00
2020-05-25 00:03:00,8534.0,8534.5,8510.0,8534.0,20076,8524.4225,500.0,235530297,2.355303,20076.0,00:03:00
2020-05-25 00:04:00,8534.0,8534.5,8534.0,8534.0,2805,8534.6078,1500.0,32868585,0.328686,2805.0,00:04:00
...,...,...,...,...,...,...,...,...,...,...,...
2020-06-01 06:55:00,9591.5,9592.0,9587.0,9587.0,1356,9591.4061,16.0,14138529,0.141385,1356.0,06:55:00
2020-06-01 06:56:00,9587.0,9587.5,9587.0,9587.0,840,9587.7277,66.0,8761777,0.087618,840.0,06:56:00
2020-06-01 06:57:00,9587.0,9587.5,9587.0,9587.5,204,9587.7277,4.0,2127722,0.021277,204.0,06:57:00


In [59]:
 broker_mapping = {
    'order_types': {
        bt.Order.Market: 'market',
        bt.Order.Limit: 'limit',
        bt.Order.Stop: 'stop',
        bt.Order.StopLimit: 'stop limit'
    },
    'mappings':{
        'closed_order':{
            'key': 'status',
            'value':'closed'
        },
        'canceled_order':{
            'key': 'result',
            'value':1}
    }
}

config = {'urls': {'api': 'https://testnet.bitmex.com'},
                   'apiKey': "PUNPhX0cNSinQ7Zv4TRAETdx",
                   'secret': "duc5b9JxLy68Oza_9cfv6I8HztwaYFSOdETWmOYOCojFtGv7",
                   'enableRateLimit': True,
                  }


In [60]:
exchange = ccxt.bitmex({
    'apiKey': 'PUNPhX0cNSinQ7Zv4TRAETdx',
    'secret': 'duc5b9JxLy68Oza_9cfv6I8HztwaYFSOdETWmOYOCojFtGv7',
})
if 'test' in exchange.urls:
    exchange.urls['api'] = exchange.urls['test'] # ←----- switch the base URL to testnet
pprint.pprint(exchange.fetch_balance())

{'BTC': {'free': 0.00847194,
         'total': 0.00892938,
         'used': 0.00045743999999999993},
 'free': {'BTC': 0.00847194},
 'info': [{'account': 319315,
           'action': '',
           'amount': 843285,
           'availableMargin': 847194,
           'commission': None,
           'confirmedDebit': 0,
           'currency': 'XBt',
           'excessMargin': 847194,
           'excessMarginPcnt': 0.434,
           'grossComm': 181349,
           'grossExecCost': 0,
           'grossLastValue': 1952200,
           'grossMarkValue': 1952200,
           'grossOpenCost': 0,
           'grossOpenPremium': 0,
           'indicativeTax': 0,
           'initMargin': 0,
           'maintMargin': 45744,
           'marginBalance': 892938,
           'marginBalancePcnt': 0.4574,
           'marginLeverage': 2.186266011749976,
           'marginUsedPcnt': 0.0512,
           'pendingCredit': 0,
           'pendingDebit': 0,
           'prevRealisedPnl': -23598,
           'prevState': '

In [61]:
store = CCXTStore(exchange="bitmex", currency="BTC", config=config, retries=5, debug=False, sandbox=True)
store.exchange = exchange

In [119]:
class MyCSVData(bt.feeds.GenericCSVData):
    params = (
        ('nullvalue', float('NaN')),
        ('dtformat', '%Y-%m-%d %H:%M:%S'),
        ('tmformat', '%H:%M:%S'),

        ('datetime', 0),
        ('time', -1),
        ('open', 1),
        ('high', 2),
        ('low', 3),
        ('close', 4),
        ('volume', 5),
        ('openinterest', 6),
    )
data2 = MyCSVData(
            dataname="FXBTUSD-1m-data.csv",
            # Do not pass values before this date
            fromdate=start_datetime,
            # Do not pass values before this date
            todate=end_datetime,
            # Do not pass values after this date
            reverse=False)

In [138]:
class TestStrategy(bt.Strategy):
    def __init__(self):
        self.next_runs = 0

    def next(self, dt=None):
        dt = dt or self.datas[0].datetime.datetime(0)
        print('%s closing price: %s' % (dt.isoformat(), self.datas[0].close[0]))
        self.next_runs += 1

In [142]:
cerebro = bt.Cerebro()
broker = store.getbroker(broker_mapping=broker_mapping)
cerebro.setbroker(broker)
cerebro.broker.setcash(100000.0)


AttributeError: 'CCXTBroker' object has no attribute 'setcash'

In [141]:
cerebro.adddata(data2)
cerebro.addstrategy(TestStrategy)
cerebro.run()

In [42]:
cerebro.run()

AttributeError: 'Plot_OldSync' object has no attribute 'mpyplot'

KeyError: 'parent'